## Webscraping de la población de CABA

Con el fin de desarrollar uno de los indicadores clave de rendimiento (KPI) para el proyecto, se requiere obtener la población de la Ciudad Autónoma de Buenos Aires. Para llevar a cabo esta tarea, se ha optado por emplear la técnica de web scraping, la cual implica acceder a una página web, examinar su código HTML y extraer de manera automatizada la información pertinente.

A continuación, se describen los pasos que se han seguido para este proceso.

In [22]:
import pandas as pd
import tools
import requests
from bs4 import BeautifulSoup


### Extracción de datos

Se especifica la dirección URL de la página de Wikipedia que alberga la tabla que presenta la información sobre la población recopilada en los censos más recientes.


In [11]:
link = "https://es.wikipedia.org/wiki/Buenos_Aires"


Se envía una solicitud HTTP para obtener el contenido de la página.


In [23]:
respuesta = requests.get(link)

Se crea un objeto BeautifulSoup para analizar el contenido HTML de la página.

In [24]:
objeto_soup = BeautifulSoup(respuesta.text, "html.parser")

Se procedió a examinar detenidamente la página que alberga la información con el objetivo de comprender la estructura de las etiquetas que conducen hasta los datos.

<img src="población - webscraping.jpg">

Se observó que la tabla de interés está encapsulada en una etiqueta "table", pero dado que existen múltiples instancias de estas etiquetas en la página, se decidió realizar una iteración por todas las etiquetas "table". Durante este proceso, se identifica un encabezado "th" que posee los atributos colspan='3' y class='navbox-title'.

Posteriormente, se verifica si este encabezado corresponde a la tabla buscada, en este caso, "Población histórica". En caso afirmativo, se procede a extraer la información de la tabla y almacenarla en una variable denominada target. Este enfoque permite localizar de manera precisa la tabla de interés dentro de la estructura más amplia de la página web.






In [29]:
# Se inicializa una variable llamada target_table como None
target_table = None

# Se buscan todas las etiquetas de tabla (<table>) en el contenido HTML
tables = objeto_soup.find_all('table')

# Se itera a través de todas las tablas encontradas en la página web
for table in tables:
    # Se busca una etiqueta de encabezado (<th>) dentro de la tabla que tenga los atributos colspan='3' y class='navbox-title'
    header = table.find('th', {'colspan': '3', 'class': 'navbox-title'})
    
    # Se comprueba si se encontró un encabezado y si el texto del encabezado contiene "Población histórica"
    if header and "Población histórica" in header.text:
        # Si se cumple la condición anterior, asigna la tabla actual a la variable target_table y sale del bucle for con break
        target_table = table
        break

En la siguiente celda, se observa la información extraída.

In [30]:
target_table

<table class="toccolours" style="width:15em;border-spacing: 0;float:right;clear:right;margin:0 0 1em 1em;"><tbody><tr><th class="navbox-title" colspan="3" style="padding:0.25em;font-size:110%">Población histórica</th></tr><tr style="font-size:95%"><th style="border-bottom:1px solid black;padding:1px;width:3em">Año</th><th style="border-bottom:1px solid black;padding:1px 2px;text-align:right"><abbr title="Población">Pob.</abbr></th><th style="border-bottom:1px solid black;padding:1px;text-align:right"><abbr title="Cambio porcentual">±%</abbr></th></tr><tr><th style="text-align:center;padding:1px">1779 </th><td style="text-align:right;padding:1px">24 205</td><td style="text-align:right;padding:1px">—    </td></tr><tr><th style="text-align:center;padding:1px">1810 </th><td style="text-align:right;padding:1px">44 800</td><td style="text-align:right;padding:1px">+85.1%</td></tr><tr><th style="text-align:center;padding:1px">1869 </th><td style="text-align:right;padding:1px">177 797</td><td s

Luego, la información contenida en la variable target es recuperada y se guarda en un DataFrame de Pandas.

In [40]:
# Comprobamos si se encontró la tabla
if target_table:
    # Creamos listas para almacenar las columnas de la tabla
    anios = []  
    poblaciones = []  
    cambios = [] 
    
    # Inicializamos una variable para almacenar el año actual
    anio_actual = None
    
    # Iteramos a través de las filas de la tabla
    for row in target_table.find_all("tr"):
        # Buscamos tanto en th (encabezados) como en td (celdas de datos)
        columns = row.find_all(["th", "td"])  
        
        # Comprobamos si la fila tiene tres columnas (Año, Población y Cambio porcentual)
        if len(columns) == 3:
            anio_actual = columns[0].get_text(strip=True)  # Obtenemos el año
            poblacion = columns[1].get_text(strip=True)  # Obtenemos la población
            cambio = columns[2].get_text(strip=True)  # Obtenemos el cambio porcentual
            anios.append(anio_actual)  # Agregamos el año a la lista anios
            poblaciones.append(poblacion)  # Agregamos la población a la lista poblaciones
            cambios.append(cambio)  # Agregamos el cambio porcentual a la lista cambios
        elif len(columns) == 2 and anio_actual:
            # Si la fila tiene dos columnas y ya tenemos el año actual, 
            # Asumimos que es la población y el cambio porcentual
            poblacion = columns[0].get_text(strip=True)  # Obtiene la población
            cambio = columns[1].get_text(strip=True)  # Obtiene el cambio porcentual
            poblaciones.append(poblacion)  # Agrega la población a la lista poblaciones
            cambios.append(cambio)  # Agrega el cambio porcentual a la lista cambios
    
    # Se crea un DataFrame de Pandas con los datos extraídos
    df_CABA = pd.DataFrame({"Año": anios, "Población": poblaciones, "Cambio porcentual": cambios})
    
    print("Ya se guardó el dataframe")
else:
    print("No se encontró la tabla deseada en la página.")

Ya se guardó el dataframe


In [41]:
df_CABA

,Año,Población,Cambio porcentual
0,Año,Pob.,±%
1,1779,24 205,—
2,1810,44 800,+85.1%
3,1869,177 797,+296.9%
4,1895,663 854,+273.4%
5,1914,1 575 814,+137.4%
6,1947,2 981 043,+89.2%
7,1960,2 966 634,−0.5%
8,1970,2 972 453,+0.2%
9,1980,2 922 829,−1.7%



El DataFrame resultante necesita algunos ajustes. Para iniciar, la primera fila, que corresponde al encabezado de la tabla, debe ser eliminada. Además, se procederá a eliminar la columna "Cambio porcentual", ya que no será utilizada. Por último, se realizará la eliminación de los espacios utilizados para separar miles en las cantidades de la columna de población.

In [42]:
# Eliminamos la primer fila
df_CABA = df_CABA.iloc[1:]
# Eliminamos la columna Cambio porcentual
df_CABA = df_CABA.drop("Cambio porcentual", axis=1)
# Sacamos los espacios de separación de miles
df_CABA["Población"] = df_CABA["Población"].str.replace(" ", "").str.replace("\xa0", "").astype(int)
df_CABA

,Año,Población
1,1779,24205
2,1810,44800
3,1869,177797
4,1895,663854
5,1914,1575814
6,1947,2981043
7,1960,2966634
8,1970,2972453
9,1980,2922829
10,1991,2965403


In [43]:
#Verificamos los tipos de datos de cada columna
tools.ver_variables(df_CABA)

,nombre_campo,tipo_datos
0,Año,[<class 'str'>]
1,Población,[<class 'int'>]


### Carga de datos limpios

In [44]:
file = "poblacion_CABA.csv"
df_CABA.to_csv(file, index=False, encoding="utf-8")
print(f"Se guardó el archivo {file}")

Se guardó el archivo poblacion_CABA.csv
